<a href="https://colab.research.google.com/github/bitratek/CNN_Stuff/blob/main/Optimized_CNNModel_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 15.3 MB/s 
     |████████████████████████████████| 1.6 MB 64.8 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
print(tf.__version__)

2.9.2


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
print(test_images[0].shape)

(28, 28)


In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
  )

  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-9-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="Mnist Fashion")

tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.10316666960716248

Best val_accuracy So Far: 0.8855000138282776
Total elapsed time: 00h 02m 56s


In [11]:
model = tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        23072     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 80)                1239120   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 1,265,082
Trainable params: 1,265,082
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2301 - accuracy: 0.9119 - val_loss: 0.3364 - val_accuracy: 0.8890
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1989 - accuracy: 0.9244 - val_loss: 0.4057 - val_accuracy: 0.8735
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1700 - accuracy: 0.9367 - val_loss: 0.3652 - val_accuracy: 0.8902
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1455 - accuracy: 0.9457 - val_loss: 0.4005 - val_accuracy: 0.8945
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1291 - accuracy: 0.9515 - val_loss: 0.4750 - val_accuracy: 0.8880
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1156 - accuracy: 0.9584 - val_loss: 0.5466 - val_accuracy: 0.8842
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1081 - accuracy: 0.9615 - val_loss: 0.6756 - val_accuracy